In [3]:
using Bloqade
using Distributions
using LegendrePolynomials
using PythonCall
using LinearAlgebra
using Printf

include("../src/BloqadeControl.jl")
using .BloqadeControl

In [ ]:
# Specify parametrization of waveforms
n_sites = 2
basis = "legendre"
n_basis = 20
use_sigmoid_envolope = true
tf = 1.0
F0 = [(x)->0.,(x)->0.]

# Specify training parameters
params = 6. * rand(Float64, (2, n_basis)) .- 3.
#params = ones(Float64, (2, n_basis))
lr = 5e-3
w_l2 = 1e-3
n_epochs = 2000
method = "adam"

# Specify parameters of Rydberg atoms
distance = 4
atoms = generate_sites(ChainLattice(), n_sites, scale=distance);
local_detuning = [-1, 1]
global_detuning = 0.0
n_samples = 20

In [ ]:
# Specify initial & target state
ket00 = Array{ComplexF64}([1, 0, 0, 0])
ket01 = Array{ComplexF64}([0, 1, 0, 0])
ket10 = Array{ComplexF64}([0, 0, 1, 0])
ket11 = Array{ComplexF64}([0, 0, 0, 1])

init_states = [ket00, ket01, ket10]
target_states = [ket00, ket01, ket11]
observables = []

for idx in 1:length(target_states)
    target_state = ArrayReg(target_states[idx]; nlevel=2)
    J = UniformScaling(1.)
    tar_op = density_matrix(target_state).state
    M = GeneralMatrixBlock(J-tar_op; nlevel=2)
    append!(observables, [M])
end

In [ ]:
# Setup optimizer
generator = WaveformGenerator(n_sites, basis, n_basis, use_sigmoid_envolope, F0, tf)
model = BloqadeModel(atoms, local_detuning, global_detuning, n_samples, init_states, observables);
optim = Optimizer(params, lr, w_l2, n_epochs, method)

In [ ]:
# Start training
update_waveform!(model, generator, optim.params)
@printf "Current loss function value is %.3f.\n" compute_loss(model, generator, optim)

train_loss!(model, generator, optim)